In [8]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pylab as plt
%matplotlib inline

In [62]:
from astroquery.simbad import Simbad
from astropy.table import Table, vstack

def read_simbad_table(obj_name):
    """Read in Astroquery results from Simbad given object name (str)"""
    result_table = Simbad.query_object(obj_name)
    return result_table

TWOPI = 2*np.pi
def get_angular_separation(ra1, dec1, ra2, dec2):
    """Compute the angular separation in radians between 2 objects given positions in degrees.
    
    Parameters
    ----------
    ra1: (float) right ascension of object 1 in degrees
    dec1: (float) declination of object 1 in degrees
    ra2: (float) right ascension of object 2 in degrees
    dec2: (float) declination of object 2 in degrees"""
    assert ra1>=0. and ra1<=TWOPI and ra2>=0. and ra2<=TWOPI, "RA must be in [0, 2pi]: {} {}".format(ra1, ra2)
    assert dec1>=-np.pi/2. and dec1<=np.pi/2. and dec2>=-np.pi/2. and dec2<=np.pi/2., "DEC must be in [-pi/2, pi/2]: {} {}".format(dec1, dec2)
    cosA = np.sin(dec1)*np.sin(dec2) + np.cos(dec1)*np.cos(dec2)*np.cos(ra1-ra2)
    return np.arccos(cosA)

In [37]:
## read in the biased draw from Aki and make 5 unbiased draws from them
Ndraw=5
obj_names_all = np.loadtxt('inputs/luvoir-A_stars.txt', usecols=(0,), delimiter=',', skiprows=1).astype(int)
obj_names = np.random.choice(obj_names_all, size=Ndraw, replace=False)
print(obj_names)

## stack each Simbad query results vertically in Astropy Table
result = read_simbad_table("Hip"+str(obj_names[0]))
for ii in range(1,len(obj_names)):
    result = vstack([result, read_simbad_table("Hip"+str(obj_names[ii]))])
    
print(result)

[ 5862 95447 40693 43726 77760]


In [42]:
## convert the Simbad RA/DEC from hh.mm.ss to degrees
from astropy import units as u
from astropy.coordinates import SkyCoord

c = SkyCoord(result['RA'], result['DEC'], unit=(u.hourangle, u.deg))

array([ 18.79634375, 291.24250083, 124.59977875, 133.57477875,
       238.16892125])

In [65]:
angular_separations = []
for ii in range(len(obj_names)-1):
    angsep = get_angular_separation(c.ra[ii].radian, c.dec[ii].radian, 
                                    c.ra[ii+1].radian, c.dec[ii+1].radian) * 180/np.pi
    angular_separations.append(angsep)
    print("Ang sep [deg] btw. HIP {} and {} = {}".format(obj_names[ii], obj_names[ii+1], 
                                           angsep))

Ang sep [deg] btw. HIP 5862 and 95447 = 96.8023853476
Ang sep [deg] btw. HIP 95447 and 40693 = 166.932074067
Ang sep [deg] btw. HIP 40693 and 43726 = 11.4130083393
Ang sep [deg] btw. HIP 43726 and 77760 = 104.418607727


In [66]:
# to compute the slew time need to know how the instrument will slew:
# take shortest path vs. along 1 axis then another...?

def get_slew_times(sep, vel=10.):
    return

In [ ]:
# fn to compute the astrometric (proper motion + parallax) signal given a set of parameters

# fn to parse the set of parameters for fiducial targets

# fn to compute the integration time/epochs needed for astrometric signal

# fn to compute slew time

# fn to minimize slew time

